In [1]:
##  code from https://github.com/dominance-analysis/dominance-analysis

In [ ]:
from brainstat import datasets
from neuromaps.nulls.spins import gen_spinsamples
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import netneurotools
from netneurotools import datasets, utils
from scipy.stats import zscore, pearsonr, ttest_ind
import seaborn as sns
from matplotlib.colors import ListedColormap
from scipy.spatial.distance import squareform, pdist
from sklearn.linear_model import LinearRegression
from statsmodels.stats.multitest import multipletests

def get_reg_r_sq(X, y):
    lin_reg = LinearRegression()
    lin_reg.fit(X, y)
    yhat = lin_reg.predict(X)
    SS_Residual = sum((y - yhat) ** 2)
    SS_Total = sum((y - np.mean(y)) ** 2)
    r_squared = 1 - (float(SS_Residual)) / SS_Total
    adjusted_r_squared = 1 - (1 - r_squared) * \
        (len(y) - 1) / (len(y) - X.shape[1] - 1)
    return adjusted_r_squared


def cv_slr_distance_dependent(X, y, coords, train_pct=.75, metric='rsq'):
    '''
    cross validates linear regression model using distance-dependent method.
    X = n x p matrix of input variables
    y = n x 1 matrix of output variable
    coords = n x 3 coordinates of each observation
    train_pct (between 0 and 1), percent of observations in training set
    metric = {'rsq', 'corr'}
    '''

    P = squareform(pdist(coords, metric="euclidean"))
    train_metric = []
    test_metric = []

    for i in range(len(y)):
        distances = P[i, :]  # for every node
        idx = np.argsort(distances)

        train_idx = idx[:int(np.floor(train_pct * len(coords)))]
        test_idx = idx[int(np.floor(train_pct * len(coords))):]

        mdl = LinearRegression()
        mdl.fit(X[train_idx, :], y[train_idx])
        if metric == 'rsq':
            # get r^2 of train set
            train_metric.append(get_reg_r_sq(X[train_idx, :], y[train_idx]))

        elif metric == 'corr':
            rho, _ = pearsonr(mdl.predict(X[train_idx, :]), y[train_idx])
            train_metric.append(rho)

        yhat = mdl.predict(X[test_idx, :])
        if metric == 'rsq':
            # get r^2 of test set
            SS_Residual = sum((y[test_idx] - yhat) ** 2)
            SS_Total = sum((y[test_idx] - np.mean(y[test_idx])) ** 2)
            r_squared = 1 - (float(SS_Residual)) / SS_Total
            adjusted_r_squared = 1-(1-r_squared)*((len(y[test_idx]) - 1) /
                                                  (len(y[test_idx]) -
                                                   X.shape[1]-1))
            test_metric.append(adjusted_r_squared)

        elif metric == 'corr':
            rho, _ = pearsonr(yhat, y[test_idx])
            test_metric.append(rho)

    return train_metric, test_metric


def get_perm_p(emp, null):
    return (1 + sum(abs(null - np.mean(null))
                    > abs(emp - np.mean(null)))) / (len(null) + 1)


def get_reg_r_pval(X, y, spins, nspins):
    emp = get_reg_r_sq(X, y)
    null = np.zeros((nspins, ))
    for s in range(nspins):
        null[s] = get_reg_r_sq(X[spins[:, s], :], y)
    return (1 + sum(null > emp))/(nspins + 1)


import warnings

import numpy as np
from tqdm import tqdm
from itertools import combinations
from scipy import optimize, spatial, special, stats as sstats
from sklearn.utils.validation import check_random_state
from sklearn.linear_model import LinearRegression
def get_dominance_stats(X, y, use_adjusted_r_sq=True, verbose=False):
    """
    Returns the dominance analysis statistics for multilinear regression.

    This is a rewritten & simplified version of [DA1]_. It is briefly
    tested against the original package, but still in early stages.
    Please feel free to report any bugs.

    Warning: Still work-in-progress. Parameters might change!

    Parameters
    ----------
    X : (N, M) array_like
        Input data
    y : (N,) array_like
        Target values
    use_adjusted_r_sq : bool, optional
        Whether to use adjusted r squares. Default: True
    verbose : bool, optional
        Whether to print debug messages. Default: False

    Returns
    -------
    model_metrics : dict
        The dominance metrics, currently containing `individual_dominance`,
        `partial_dominance`, `total_dominance`, and `full_r_sq`.
    model_r_sq : dict
        Contains all model r squares

    Notes
    -----
    Example usage

    .. code:: python

        from netneurotools.stats import get_dominance_stats
        from sklearn.datasets import load_boston
        X, y = load_boston(return_X_y=True)
        model_metrics, model_r_sq = get_dominance_stats(X, y)

    To compare with [DA1]_, use `use_adjusted_r_sq=False`

    .. code:: python

        from dominance_analysis import Dominance_Datasets
        from dominance_analysis import Dominance
        boston_dataset=Dominance_Datasets.get_boston()
        dominance_regression=Dominance(data=boston_dataset,
                                       target='House_Price',objective=1)
        incr_variable_rsquare=dominance_regression.incremental_rsquare()
        dominance_regression.dominance_stats()

    References
    ----------
    .. [DA1] https://github.com/dominance-analysis/dominance-analysis

    """

    # this helps to remove one element from a tuple
    def remove_ret(tpl, elem):
        lst = list(tpl)
        lst.remove(elem)
        return tuple(lst)

    # sklearn linear regression wrapper
    def get_reg_r_sq(X, y):
        lin_reg = LinearRegression()
        lin_reg.fit(X, y)
        yhat = lin_reg.predict(X)
        SS_Residual = sum((y - yhat) ** 2)
        SS_Total = sum((y - np.mean(y)) ** 2)
        r_squared = 1 - (float(SS_Residual)) / SS_Total
        adjusted_r_squared = 1 - (1 - r_squared) * \
            (len(y) - 1) / (len(y) - X.shape[1] - 1)
        if use_adjusted_r_sq:
            return adjusted_r_squared
        else:
            return r_squared

    # generate all predictor combinations in list (num of predictors) of lists
    n_predictor = X.shape[-1]
    # n_comb_len_group = n_predictor - 1
    predictor_combs = [list(combinations(range(n_predictor), i))
                       for i in range(1, n_predictor + 1)]
    if verbose:
        print(f"[Dominance analysis] Generated \
              {len([v for i in predictor_combs for v in i])} combinations")

    # get all r_sq's
    model_r_sq = dict()
    for len_group in tqdm(predictor_combs, desc='num-of-predictor loop',
                          disable=not verbose):
        for idx_tuple in tqdm(len_group, desc='insider loop',
                              disable=not verbose):
            r_sq = get_reg_r_sq(X[:, idx_tuple], y)
            model_r_sq[idx_tuple] = r_sq
    if verbose:
        print(f"[Dominance analysis] Acquired {len(model_r_sq)} r^2's")

    # getting all model metrics
    model_metrics = dict([])

    # individual dominance
    individual_dominance = []
    for i_pred in range(n_predictor):
        individual_dominance.append(model_r_sq[(i_pred,)])
    individual_dominance = np.array(individual_dominance).reshape(1, -1)
    model_metrics["individual_dominance"] = individual_dominance

    # partial dominance
    partial_dominance = [[] for _ in range(n_predictor - 1)]
    for i_len in range(n_predictor - 1):
        i_len_combs = list(combinations(range(n_predictor), i_len + 2))
        for j_node in range(n_predictor):
            j_node_sel = [v for v in i_len_combs if j_node in v]
            reduced_list = [remove_ret(comb, j_node) for comb in j_node_sel]
            diff_values = [
                model_r_sq[j_node_sel[i]] - model_r_sq[reduced_list[i]]
                for i in range(len(reduced_list))]
            partial_dominance[i_len].append(np.mean(diff_values))

    # save partial dominance
    partial_dominance = np.array(partial_dominance)
    model_metrics["partial_dominance"] = partial_dominance
    # get total dominance
    total_dominance = np.mean(
        np.r_[individual_dominance, partial_dominance], axis=0)
    # test and save total dominance
    assert np.allclose(total_dominance.sum(),
                       model_r_sq[tuple(range(n_predictor))]), \
           "Sum of total dominance is not equal to full r square!"
    model_metrics["total_dominance"] = total_dominance
    # save full r^2
    model_metrics["full_r_sq"] = model_r_sq[tuple(range(n_predictor))]

    return model_metrics, model_r_sq


In [57]:
"""
set-up
"""

path = 'your pathway/MIND/outcome/'

# set up parcellation
# coords_DK308_hemi
hemiid = np.genfromtxt(path+'coords_DK308_hemi.txt')
coords =  np.genfromtxt(path+'coords_DK308.txt')
nnodes = 308
nspins = 10000
spins = gen_spinsamples(coords, hemiid, n_rotate=nspins, seed=1234)

In [30]:
# load the receptor data (from neuromaps: https://netneurolab.github.io/neuromaps/usage.html)
receptor_data = np.genfromtxt(path+'neuromaps_receptors_scaled_DK308.csv', delimiter=',')
receptor_names = np.loadtxt(path+'nuromaps_receptor_names.txt', dtype='str')
MIND_Dementia = np.genfromtxt(path+'final_Tmap_NC_Wu_NIFD_all_DK308.csv', delimiter=',')
#MIND_Dementia = MIND_Dementia[:, 1:]
disorders =  ["FTD", "FTD-rep", "bvFTD", "bvFTD-rep", "svPPA", "svPPA-rep", "nfvPPA", "NC", "NC-rep"];

# colourmaps
cmap = np.genfromtxt(path+'colourmap.csv', delimiter=',')
cmap_div = ListedColormap(cmap)
cmap_seq = ListedColormap(cmap[128:, :])

In [58]:
# load the psychiatry data (from ENIGMA)
receptor_data = np.genfromtxt(path+'psychiatry_6_scaled_DK68.csv', delimiter=',')
receptor_names = np.loadtxt(path+'psychiatry_6_scaled_name.txt', dtype='str')
MIND_Dementia = np.genfromtxt(path+'final_Tmap_NC_Wu_NIFD_all_DK68.csv', delimiter=',')
#MIND_Dementia = MIND_Dementia[:, 1:]
disorders =  ["FTD", "FTD-rep", "bvFTD", "bvFTD-rep", "svPPA", "svPPA-rep", "nfvPPA"];

# colourmaps
cmap = np.genfromtxt(path+'colourmap.csv', delimiter=',')
cmap_div = ListedColormap(cmap)
cmap_seq = ListedColormap(cmap[128:, :])

In [59]:
print(receptor_data.shape)
print(receptor_names)
print(MIND_Dementia.shape)
print(disorders)
#print(receptor_data)
print(len(disorders))

(68, 6)
['ADHD' 'ASD' 'BIPO' 'MDD' 'OCD' 'SZ']
(68, 9)
['FTD', 'FTD-rep', 'bvFTD', 'bvFTD-rep', 'SD', 'SD-rep', 'PNFA']
7


In [60]:
"""
Dominance analysis
"""

model_metrics = dict([])
train_metric = np.zeros([nnodes, len(disorders)])
test_metric = np.zeros(train_metric.shape)
model_pval = np.zeros((len(disorders), ))

for i in range(len(disorders)):
    print(i)
    m, _ = get_dominance_stats(receptor_data,
                                     zscore(MIND_Dementia[:, i]))
    model_metrics[disorders[i]] = m
    # cross validate the model
    train_metric[:, i], test_metric[:, i] = \
        cv_slr_distance_dependent(receptor_data, zscore(MIND_Dementia[:, i]),
                                  coords, .75, metric='corr')
    # get p-value of model
    model_pval[i] = get_reg_r_pval(receptor_data,
                                   zscore(MIND_Dementia[:, i]), 
                                   spins, nspins)

model_pval = multipletests(model_pval, method='fdr_bh')[1]

dominance = np.zeros((len(disorders), len(receptor_names)))

0
1
2
3
4
5
6


In [ ]:
from statsmodels.stats.multitest import fdrcorrection

# FDR Correction
model_pval_corrected = fdrcorrection(model_pval)[1]

print(model_pval_corrected)

In [ ]:
for i in range(len(model_metrics)):
    tmp = model_metrics[disorders[i]]
    dominance[i, :] = tmp["total_dominance"]
np.save(path+'results/dominance_enigma.npy', dominance)
np.save(path+'results/enigma_cv_train.npy', train_metric)

plt.ion()
plt.figure()
plt.bar(np.arange(len(disorders)), np.sum(dominance, axis=1),
        tick_label=disorders)
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.savefig(path+'figures/bar_dominance.eps')

dominance[np.where(model_pval_corrected >= 0.05)[0], :] = 0
plt.ion()
plt.figure()
sns.heatmap(dominance / np.sum(dominance, axis=1)[:, None],
            xticklabels=receptor_names, yticklabels=disorders,
            cmap=cmap_seq, linewidth=.5)
plt.tight_layout()
plt.savefig(path+'figures/heatmap_dominance.eps')


In [ ]:
print(model_pval)
print(dominance)
print(np.sum(dominance, axis=1))
print(dominance / np.sum(dominance, axis=1)[:, None])
PVALUE = model_pval
ADJUSTR = np.sum(dominance, axis=1)
PERCENT = 100*(dominance / np.sum(dominance, axis=1)[:, None])